# Read CSV from laptop

In [ ]:
import pandas as pd

df = pd.read_csv("healthcare-dataset-stroke-data.csv")
df.head()

# Data Cleaning

## bmi

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(subset=['bmi'], inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4)) #define a space to plot
df[['bmi']].boxplot(grid=False, rot=45, fontsize=18) #rotate the labels
plt.title("BMI", fontsize=14)
plt.show()


In [ ]:
df.describe()

In [ ]:
q3 = df['bmi'].quantile(0.75)
q1 = df['bmi'].quantile(0.25)
iqr = q3 - q1
outlier_max = q3 + 1.5*iqr
outlier_min = q1 - 1.5*iqr

print("Extreme Value (Min): ", outlier_min)
print("Extreme Value (Max): ", outlier_max)

In [ ]:
df[df["bmi"] > outlier_max]

In [ ]:
df = df[df["bmi"] <= outlier_max]

In [ ]:
df.describe()

- The missing values, found in the bmi column, are removed as we cannot assume the value of bmi.
- Mild outliers in bmi column are also removed.

## Gender

In [ ]:
df.nunique()

In [ ]:
df['gender'].unique()

In [ ]:
df[df["gender"] == 'Other']

In [ ]:
df.drop([3116],inplace=True)

In [ ]:
df['gender'].unique()

In [ ]:
df.nunique()

- The gender column has three different values - Male, Female and Others.
- However, according to scientific terms, there are only 2 types of gender - Male and Female.
- As we are unsure whether Other is classified either Male or Female, the row that contained Other as gender was removed.

## Age

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(6,4)) #define a space to plot
df[['age']].boxplot(grid=False, rot=45, fontsize=18) #rotate the labels
plt.title("Age", fontsize=14)
plt.show()

In [ ]:
df[df["age"] == 82]

In [ ]:
df['stroke'].value_counts().plot(kind='pie',autopct='%1.0f%%')

- No cleaning would be required for age as there is no missing value or extreme values.

## Gender - Transformation

In [ ]:
df['gender'].value_counts().plot(kind='pie',autopct='%1.0f%%')

In [ ]:
def myfunc(x):
    if x == 'Female':
        return 0
    else:
        return 1

df_gender=df.assign(gender_code = lambda x:df['gender'].apply(myfunc))
                                                                                  
#preview
df_gender.tail()

In [ ]:
del df_gender['gender']

In [ ]:
df_gender.tail()

- 0 represents Female
- 1 represents Male

## Ever_married

In [ ]:
df_gender['ever_married'].value_counts().plot(kind='pie',autopct='%1.0f%%')

In [ ]:
df_married=df_gender.assign(is_married = ~(df_gender['ever_married'].str.contains('No', case=False))) # checking if its NOT 'No'
                                                                                  
#preview
df_married.tail()

In [ ]:
del df_married['ever_married']

In [ ]:
df_married.tail()

## Residence_type

In [ ]:
df['Residence_type'].value_counts().plot(kind='pie',autopct='%1.0f%%')

In [ ]:
df_married['Residence_type'].unique()

In [ ]:
def myfunc(x):
    if x == 'Rural':
        return 0
    else:
        return 1

df_residence=df_married.assign(residence_type = lambda x:df_married['Residence_type'].apply(myfunc))
                                                                                  
#preview
df_residence.tail()

In [ ]:
del df_residence['Residence_type']

In [ ]:
df_residence.tail()

- 0 represents Rural residency
- 1 represents Urban residency

## Smoking_status - Evaluation

In [ ]:
df_residence.smoking_status.value_counts().plot(kind='bar', title='Smoke Type')

In [ ]:
df_smoke = df_residence[df_residence['smoking_status'] != 'Unknown']

In [ ]:
df_smoke['smoking_status'].unique()

In [ ]:
df_smoke.hist()

## Work_type

In [ ]:
df_smoke.work_type.value_counts().plot(kind='bar', title='Work Type')

In [ ]:
df_smoke.work_type.value_counts()

In [ ]:
df_updated = pd.get_dummies(df_smoke) #auto one-hot encoding

In [ ]:
del df_updated['id']
df_updated.head()

In [ ]:
df_updated.info()

In [ ]:
corr_mat = df_updated.corr()
corr_mat.style.background_gradient(cmap="Reds")

# Data Sampling & Scaling

In [ ]:
df_updated[['hypertension', 'stroke', 'heart_disease']].hist()

In [ ]:
from imblearn.over_sampling import RandomOverSampler

target = df_updated['stroke']
df_updated = df_updated.drop(columns='stroke')

ros = RandomOverSampler(random_state=0)
X, y = ros.fit_resample(df_updated, target)

In [ ]:
X[['hypertension', 'heart_disease']].hist()


In [ ]:
y.hist()
plt.title("Stroke column (y)")

In [ ]:
X.describe()

# Machine Learning Algorithms

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=5)

- The variable “y” represents the stroke column
- On the other hand, the variable “X” represents the other columns in the dataset.

- Dataset is split into 70% train and 30% test.
- 4 return values from calling the train_test_split() function.
- Random_state=5 allows us to all have the same reproducible effect. (to ensure that the train/test values are the same)


## Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_reg = DecisionTreeClassifier(random_state=1) #, max_depth=10
dt_reg.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

print(accuracy_score(y_test, dt_reg.predict(X_test)))

print(f1_score(y_test, dt_reg.predict(X_test)))

## Gradient Boosting Classifier

In [ ]:
from sklearn import ensemble

gbc = ensemble.GradientBoostingClassifier(random_state=1)

gbc.fit(X_train,y_train)

In [ ]:
print(accuracy_score(y_test, gbc.predict(X_test)))

print(f1_score(y_test, gbc.predict(X_test)))

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=1)
lr.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

print(accuracy_score(y_test, lr.predict(X_test)))

print(f1_score(y_test, lr.predict(X_test)))

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=1)
rfc.fit(X_train, y_train)

In [ ]:
print(accuracy_score(y_train, rfc.predict(X_train)))

print(accuracy_score(y_test, rfc.predict(X_test)))

print(f1_score(y_test, rfc.predict(X_test)))

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rfc, X, y)
print(scores)

print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

## k-Nearest Neighbours Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

In [ ]:
print(accuracy_score(y_test, knn.predict(X_test)))

print(f1_score(y_test, knn.predict(X_test)))

- From the models shown above, Random Forest Classifier is seemed to be the best model among the other 4 models as it has the best accuracy score and f1 score.
- Better accuracy score and f1 score means that the model is precised and accurate in predicting the classified values.

# Features Selection

In [ ]:
X.info()

In [ ]:
import numpy as np
import joblib

# These are the feature labels from our data set
feature_labels = np.array(['age','hypertension','heart_disease','avg_glucose_level','bmi','gender_code','is_married','residence_type','work_type_Govt_job','work_type_Never_worked','work_type_Private','work_type_Self-employed','work_type_children','smoking_status_formerly smoked', 'smoking_status_never smoked', 'smoking_status_smokes'])

# Create a numpy array based on the model's feature importances
importance = rfc.feature_importances_

# Sort the feature labels based on the feature importance rankings from the model
feature_indexes_by_importance = importance.argsort()

# Print each feature label, from most important to least important (reverse order)
for index in feature_indexes_by_importance:
    print("{} - {:.2f}%".format(feature_labels[index], (importance[index] * 100.0)))

From features selection, the following columns will be kept:
- age
- hypertension
- heart_disease
- avg_glucose_level
- bmi
- gender_code
- is_married
- residence_type
- smoking_status never smoked

In [ ]:
X = X.drop(columns=['work_type_Never_worked', 'work_type_children', 'work_type_Govt_job', 'smoking_status_smokes', 'smoking_status_formerly smoked', 'work_type_Private', 'work_type_Self-employed'])

In [ ]:
X.head()

In [ ]:
y.describe()

In [ ]:
from sklearn.model_selection import train_test_split

y = y.to_numpy()
X = X.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=5)

# Model Prediction

In [ ]:
import joblib

# Load the model we trained previously
model_ensemble = rfc.fit(X_train, y_train)

# For the house we want to value, we need to provide the features in the exact same
# arrangement as our training data set.
brain_to_value = [
    75, #age
    1, #hypertension 
    1, #heart_disease
    200, #avg_glucose_level
    45, #bmi
    1, #gender_code
    1, #is_married
    0, #residence_type
    0 #never_smoked

]

# scikit-learn assumes you want to predict the values for lots of houses at once, so it expects an array.
# We just want to look at a single house, so it will be the only item in our array.
stroke_to_value = [
    brain_to_value
]

# Run the model and make a prediction for each house in the homes_to_value array
predicted_stroke_values = model_ensemble.predict(stroke_to_value)

# Since we are only predicting the price of one house, just look at the first prediction returned
predicted_value = predicted_stroke_values[0]

#print("Chances of you getting stroke is %.2f " % (predicted_value*100) + "%")

if predicted_value == 1:
    print("WARNING! HIGH CHANCE! You might have stroke in the future... Pls monitor your health!")
else:
    print("Your chances of getting stroke is low. Pls continue to practice good health management!")

In [ ]:
import joblib

joblib.dump(rfc,'models/rfc_trained.pkl')